In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import make_classification
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import cross_val_score

In [2]:
x_data_generated, y_data_generated = make_classification(scale=1)
df = pd.DataFrame(x_data_generated)
df = pd.concat([df, pd.Series(y_data_generated)], axis = 1)
df_corr = df.corr(method = 'spearman')
accuracy = cross_val_score(LogisticRegression(), x_data_generated, y_data_generated, scoring='accuracy').mean()
count_1 = x_data_generated.shape[1]
print("Средняя точность модели логистической регрессии:", round(accuracy, 2))

Средняя точность модели логистической регрессии: 0.88


In [3]:
upper_tri = df_corr.where(np.triu(np.ones(df_corr.shape), k=1).astype(bool))
columns_with_max_corr = [column for column in upper_tri.columns if any(abs(upper_tri[column]) > 0.5)]
accuracy_after_feature_selection_with_corr_matrix = cross_val_score(LogisticRegression(), x_data_generated[:, columns_with_max_corr], y_data_generated, scoring='accuracy').mean()
count_2 = x_data_generated[:, columns_with_max_corr].shape[1]
print("Средняя точность после выбора признаков на основе матрицы корреляции:", round(accuracy_after_feature_selection_with_corr_matrix, 2))

Средняя точность после выбора признаков на основе матрицы корреляции: 0.94


In [11]:
accuracy_after_drop_low_cov_feature = cross_val_score(LogisticRegression(), VarianceThreshold(threshold=0.5).fit_transform(x_data_generated), y_data_generated, scoring='accuracy').mean()
count_3 = VarianceThreshold(threshold=0.5).fit_transform(x_data_generated).shape[1]
print("Средняя точность после отсева низковариативных признаков:", round(accuracy_after_drop_low_cov_feature, 2))

Средняя точность после отсева низковариативных признаков: 0.88


In [5]:
accuracy_after_anova = cross_val_score(LogisticRegression(), SelectKBest(f_classif, k=5).fit_transform(x_data_generated, y_data_generated), y_data_generated, scoring='accuracy').mean()
count_4 = SelectKBest(f_classif, k=5).fit_transform(x_data_generated, y_data_generated).shape[1]
print("Средняя точность после отбора признаков на основе дисперсионного анализа:", round(accuracy_after_anova, 2))

Средняя точность после отбора признаков на основе дисперсионного анализа: 0.94


In [6]:
accuracy_after_feature_selection_lr = cross_val_score(LogisticRegression(), SelectFromModel(LogisticRegression(penalty="l1", solver='liblinear')).fit_transform(x_data_generated, y_data_generated), y_data_generated, scoring='accuracy').mean()
count_5 = SelectFromModel(LogisticRegression(penalty="l1", solver='liblinear')).fit_transform(x_data_generated, y_data_generated).shape[1]
print("Средняя точность после отбора признаков с помощью логистической регрессии (L1 регуляризация):", round(accuracy_after_feature_selection_lr, 2))

Средняя точность после отбора признаков с помощью логистической регрессии (L1 регуляризация): 0.9


In [7]:
accuracy_after_feature_selection_rf = cross_val_score(LogisticRegression(), x_data_generated[:, RandomForestClassifier().fit(x_data_generated, y_data_generated).feature_importances_.argsort()[::-1][:10]], y_data_generated, scoring='accuracy').mean()
count_6 = x_data_generated[:, RandomForestClassifier().fit(x_data_generated, y_data_generated).feature_importances_.argsort()[::-1][:10]].shape[1]
print("Средняя точность после отбора признаков с помощью модели RandomForest:", round(accuracy_after_feature_selection_rf, 2))

Средняя точность после отбора признаков с помощью модели RandomForest: 0.91


In [8]:
sfs_forward = SequentialFeatureSelector(LogisticRegression(), n_features_to_select=10, direction="forward")
accuracy_after_forward_sfs = cross_val_score(LogisticRegression(), sfs_forward.fit_transform(x_data_generated, y_data_generated), y_data_generated, scoring='accuracy').mean()
count_7 = sfs_forward.fit_transform(x_data_generated, y_data_generated).shape[1]
print("Средняя точность после перебора признаков с помощью прямого SequentialFeatureSelector:", round(accuracy_after_forward_sfs, 2))

Средняя точность после перебора признаков с помощью прямого SequentialFeatureSelector: 0.94


In [9]:
sfs_backward = SequentialFeatureSelector(LogisticRegression(), n_features_to_select=10, direction="backward")
accuracy_after_backward_sfs = cross_val_score(LogisticRegression(), sfs_backward.fit_transform(x_data_generated, y_data_generated), y_data_generated, scoring='accuracy').mean()
count_8 = sfs_backward.fit_transform(x_data_generated, y_data_generated).shape[1]
print("Средняя точность после перебора признаков с помощью обратного SequentialFeatureSelector:", round(accuracy_after_backward_sfs, 2))

Средняя точность после перебора признаков с помощью обратного SequentialFeatureSelector: 0.93


In [12]:
pd.set_option("max_colwidth", 500)
relult = (pd.DataFrame({'Способ выбора признаков': ['Взяли признаки по дефолту', 'На основе матрицы корреляции', 'Отсекли низковариативные признаки', 'На основе дисперсионного анализа', 'C помощью логистической регрессии (L1 регуляризация)', 'C помощью модели RandomForest', 'C помощью прямого SequentialFeatureSelector', 'C помощью обратного SequentialFeatureSelector'], 
                        'Количество признаков': [count_1, count_2, count_3, count_4, count_5, count_6, count_7, count_8], 
                        'Средняя точность модели': [accuracy, accuracy_after_feature_selection_with_corr_matrix, accuracy_after_drop_low_cov_feature, accuracy_after_anova, accuracy_after_feature_selection_lr, accuracy_after_feature_selection_rf, accuracy_after_forward_sfs, accuracy_after_backward_sfs]}))
relult

,Способ выбора признаков,Количество признаков,Средняя точность модели
0,Взяли признаки по дефолту,20,0.88
1,На основе матрицы корреляции,3,0.94
2,Отсекли низковариативные признаки,20,0.88
3,На основе дисперсионного анализа,5,0.94
4,C помощью логистической регрессии (L1 регуляризация),11,0.90
5,C помощью модели RandomForest,10,0.91
6,C помощью прямого SequentialFeatureSelector,10,0.94
7,C помощью обратного SequentialFeatureSelector,10,0.93
